# Decision Trees

**What does it do?**
A Decision Tree is a supervised learning algorithm used for both classification and regression tasks. It works by learning simple decision rules inferred from the data features. Think of it like a flowchart where each internal node represents a decision on a feature, each branch is the outcome of that decision, and each leaf node represents a final prediction (class or value).

**How it works (Classification example):**

1. Pick the best feature to split the data. This is based on metrics like Gini Impurity, Entropy, or Information Gain.

2. Split the dataset based on that feature.

3. Repeat this process recursively for each child node until:

- All data points in a node belong to the same class, or

- A stopping condition is met (e.g. max depth, min samples).

- Assign class labels to leaf nodes (majority vote).

**Key Concepts:**

- Gini Impurity: Measures how “mixed” the classes are in a node. Lower is better.

- Entropy: A measure of disorder. Decision Trees using entropy try to maximize information gain.

- Information Gain: The reduction in entropy before vs after a split.

- Overfitting: Trees can grow too deep and perfectly fit the training data, hurting generalization. Use pruning or set depth limits.

- Pruning: Cutting back the tree to prevent overfitting (pre-pruning or post-pruning).

**Decision Tree Characteristics:**

- Interpretable: Easy to visualize and understand.

- Non-parametric: No assumptions about data distribution.

- Can handle both numerical and categorical features.

- Sensitive to small changes in data (can lead to different splits).



**Popular Variants:**

- Random Forests – ensemble of Decision Trees for better performance.

- Gradient Boosted Trees – builds trees sequentially, each correcting the last.







![Alt text](../images/decision_tree_formula.png)

# Implmentation without sklearn

In [1]:
# Load data and test
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [2]:
# Tree Node
class Node:
    def __init__(self, feature=None, value=None, left=None, right=None, label=None):
        self.feature = feature
        self.value = value
        self.left = left
        self.right = right
        self.label = label

In [14]:
import numpy as np 

# Calculate Gini Impurity
def gini(y):
    classes = np.unique(y)
    probs = [(np.sum(y == c) / len(y)) for c in classes]
    return 1 - sum(p**2 for p in probs)


# Calculate Entropy Impurity
def entropy(y): 
    classes = np.unique(y)
    probs = [(np.sum(y == c) / len(y)) for c in classes]
    return  - np.sum(probs * np.log(probs))


In [15]:
# Split data by feature and value
def split(X, y, feature, value):
    left = X[:, feature] <= value
    right = X[:, feature] > value
    return X[left], y[left], X[right], y[right]

# Find best split
def best_split(X, y, method="gini"):
    best_score, best_feat, best_val = 1, None, None
    for feat in range(X.shape[1]):
        for val in np.unique(X[:, feat]):
            X_l, y_l, X_r, y_r = split(X, y, feat, val)
            if len(y_l) == 0 or len(y_r) == 0:
                continue
            if method == "gini": 
                score = (len(y_l)*gini(y_l) + len(y_r)*gini(y_r)) / len(y)
            else: 
                score = (len(y_l)*entropy(y_l) + len(y_r)*entropy(y_r)) / len(y)
            if score < best_score:
                best_score, best_feat, best_val = score, feat, val
    return best_feat, best_val

# Get majority label
def majority_label(y):
    labels, counts = np.unique(y, return_counts=True)
    return labels[np.argmax(counts)]

# Build tree
def build_tree(X, y, depth=0, max_depth=3):
    if len(np.unique(y)) == 1 or depth == max_depth:
        return Node(label=majority_label(y))
    feat, val = best_split(X, y, method="entropy")
    if feat is None:
        return Node(label=majority_label(y))
    X_l, y_l, X_r, y_r = split(X, y, feat, val)
    return Node(feature=feat, value=val,
                left=build_tree(X_l, y_l, depth+1, max_depth),
                right=build_tree(X_r, y_r, depth+1, max_depth))

# Predict one sample
def predict_one(x, node):
    if node.label is not None:
        return node.label
    if x[node.feature] <= node.value:
        return predict_one(x, node.left)
    else:
        return predict_one(x, node.right)

# Predict many
def predict(tree, X):
    return [predict_one(x, tree) for x in X]



tree = build_tree(X_train, y_train, max_depth=3)
y_pred = predict(tree, X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.9


# 🌲 Random Forest Steps: 

1. Bootstrap Sampling: Randomly sample (with replacement) from the training data to create multiple datasets.

2. Build Decision Trees

3. Train a decision tree on each bootstrap sample.

    - At each split, only a random subset of features is considered (adds randomness and reduces overfitting).

    - Trees are typically grown deep (not pruned), making them strong individual classifiers.

4. Aggregate Predictions

    - Classification: majority vote across trees

    - Regression: average the outputs


In [18]:
# Random Forest: build multiple trees on random subsets of data and features
import random

def bootstrap_sample(X, y):
    indices = np.random.choice(len(X), len(X), replace=True)
    return X[indices], y[indices]

def random_forest(X, y, n_trees=5, max_depth=3):
    trees = []
    for _ in range(n_trees):
        X_sample, y_sample = bootstrap_sample(X, y)
        tree = build_tree(X_sample, y_sample, max_depth=max_depth)
        trees.append(tree)
    return trees

def predict_forest(trees, X):
    tree_preds = np.array([predict(tree, X) for tree in trees])
    final_preds = []
    for i in range(X.shape[0]):
        vals, counts = np.unique(tree_preds[:, i], return_counts=True)
        final_preds.append(vals[np.argmax(counts)])
    return np.array(final_preds)


# Random Forest
forest = random_forest(X_train, y_train, n_trees=10, max_depth=3)
y_forest_pred = predict_forest(forest, X_test)
print("Accuracy (Random Forest):", accuracy_score(y_test, y_forest_pred))


Accuracy (Random Forest): 0.9333333333333333


# ⚡ Gradient Boosting

**🎯 Key Idea**: Build the model sequentially, each step reducing the error from the previous step.

**Steps**: 
1. Start with a Weak Model: Usually a simple decision tree (a "stump") that makes an initial prediction, or a constant average value. 2.
2. Repeat: 
    
    1. Compute Residuals (Errors): Measure how far off the model’s predictions are from the actual targets.
    
    2. Train Next Tree on Residuals: A new tree is trained to predict the residuals (errors) — i.e., where the last model failed.
    
    3. Update the Model: Add the new tree’s predictions to the previous ones, scaled by a learning rate (to control step size): F = F_old + $\alpha$ * F_new

Continue adding trees, each one correcting the mistakes of the combined ensemble so far.



# 🚀 XGBoost (Extreme Gradient Boosting)
XGBoost is a fast, regularized, and optimized version of Gradient Boosting. 

It is designed for handling large and complex datasets. Has regularization and pruning

**Steps:**

1. Start with an initial prediction, often the mean of the target.
2. Repeat:
    1. Calculate the residuals = prediction - target
    2. Calculate the similarity score = (sum of residuals)<sup>2</sup> / (N(residuals) + $\lambda$)
    3. Find the best split with the most gain: Gain = left_child_similarity + right_child_similarity - root_similarity
    4. If the gain is more than gamma do the split. 
3. The finaloutput prediction is = (sum of residuals) / (N(residuals) + $\lambda$)


**Source**: StatQuest YouTube video: https://www.youtube.com/watch?v=OtD8wVaFm6E

# ⚡ LightGBM
LightGBM is a fast, efficient gradient boosting framework developed by Microsoft, optimized for speed and scalability.

How it works: 
1. Histogram-based Binning
LightGBM discretizes continuous features into bins (e.g., 255 bins). This speeds up training and reduces memory usage.

2. Leaf-wise Tree Growth:
LightGBM does not grow level-wise, instead, it chooses the leaf with the max loss (or residual) to split. So, it can grow deep, and unbalanced trees.
It can overfit easily if you don't set a proper stopping condition for it. 


# Implmentation of decision tree with sklearn

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load data
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Train model
clf = DecisionTreeClassifier(max_depth=3, random_state=0)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9666666666666667
